# Shakespeare play text prediction

Jan Polzer and Ryan Duckworth

Dataset from: https://www.kaggle.com/kingburrito666/shakespeare-plays

In [1]:
import numpy as np
import pandas as pd
import random

Load dataset and drop the rows where at least one element is missing.

In [2]:
text = pd.read_csv('data/Shakespeare_data.csv').dropna()

We imported 105,152 lines of Shakespeare text.

In [3]:
print(text.shape)
text.head(1)

(105152, 6)


,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"


The PlayerLine column contains the text we want to use.

In [4]:
text = text.iloc[:,-1].values
print(text)

['So shaken as we are, so wan with care,'
 'Find we a time for frighted peace to pant,'
 'And breathe short-winded accents of new broils' ...
 'Each one demand an answer to his part'
 "Perform'd in this wide gap of time since first"
 "We were dissever'd: hastily lead away."]


We join the text using the new line character.

In [5]:
data = "\n".join(text)

In [6]:
dictionary = {}

We create a dictionary and count the number of times the next word occurs after the current word.

In [7]:
for line in data.split('\n'):
    words = line.split(' ')
    for word1, word2 in zip(words[:-1],words[1:]):
        if word1 not in dictionary:
            dictionary[word1] = {word2:1}
        else:
            if word2 not in dictionary[word1]:
                #First time encountered word2 after word1 (not in dictionary), set to 1 occurrence
                dictionary[word1][word2] = 1
            else:
                #Additional times encountered word2 after word1, increment the number of occurrences
                dictionary[word1][word2] += 1

Predict next word function. word1 is the previous word. The most_frequent flag has more useful results when set to False, to avoid repetition of the most frequently used word. The randomization of results works much better.

In [8]:
def predict_next_word(word1, dictionary, most_frequent):
    if word1 not in dictionary:
        word1 = ""
    
    path = dictionary[word1]
    pathArray = []
    
    if most_frequent:
        #Return the most frequently occuring word that occurs after last_word
        most_frequent_word = max(path, key=path.get)
        return most_frequent_word
    else:
        #Return a random word that occurs after last_word
        for word in path:
            freq = path[word]
            for f in range(freq):
                pathArray.append(word)
        return pathArray[random.randint(0, len(pathArray)-1)]

Generate new text from the text corpus


In [9]:
sentence = ""
last_word = ""
length = 100

for each_word in range(0, length):
    next_word = predict_next_word(last_word, dictionary, False)
    sentence += " %s"%(next_word)
    last_word = next_word
print(sentence.strip())

O, no judgment, your best of her, I am half his manage, this I have to sleep. Sir, I must be hanged, and master were enwombed mine: and, for bacon, I have with me, but mark but I would it seems hid, an argosy? Well, sit and of our wrath? How many may pick a man, yet too much: brings down from her well, another time cohered with such a prince shall we have more than thy grief, and breaking here, here young Dumain, a newer object I will to bring a serpent's poison. Say, gentlemen, we make me himself


Perform text prediction given a sequence of words

In [10]:
def predict_given_words(words, length, most_frequent = False):
    
    w = words.split(' ')
    last_word = w[-1]

    for each_word in range(0, length):
        next_word = predict_next_word(last_word, dictionary, most_frequent)
        words += " %s"%(next_word)
        last_word = next_word
        words.strip()
    return words

3 Examples below of predicting the next word using random words found in the dictionary after the last word

In [11]:
sentence = predict_given_words("I shall be", 50)
print(sentence)

I shall be thyself! Speak England in thy mind, I'll die this place in a note and this way. Let me so. My good my power to him, for kissing in justice ward to live to be cramm'd up into mine uncle: if you common or at hand. What is dead! I can:


In [12]:
sentence = predict_given_words("O my", 50)
print(sentence)

O my breathing, no woman. That many hours, and not here. How many time of woe, I go hard thoughts: I thank your pie is the like, methought, what he were probable. By my lord. Come now, Adam! no less nor Lucius general as that. Go you long: good lord, dear Cassio!


In [17]:
sentence = predict_given_words("As this", 50)
print(sentence)

As this country servant-maid But in the father found good fellow, I think you? You have accosted her, I shall do aspire to, away! I cannot call in destiny: 'that oft,' says the captains look thou thyself a passage to the throne of fair Mistress Shore. Some proper squire! And men feel


Example below of using most frequent word found in the in dictionary after the last word
We found the randomization above to be better for generating text because using the most frequent words creates a repetitive pattern. See 2nd example below for example: "and the king..."

In [14]:
sentence = predict_given_words("I shall be", 5, True)
print(sentence)

I shall be a man of the king


In [15]:
#Example of the repetition we don't want when only using the most frequent words, without randomization
sentence = predict_given_words("I shall be", 11, True)
print(sentence)

I shall be a man of the king and the king and the king


In [20]:
#By passing False into the most_frequent parameter of the predict_given_words and predict_next_word functions, 
#it produces much better text using randomization of words in the path of the previous word.
sentence = predict_given_words("I shall be", 11, False)
print(sentence)

I shall be like bright swords, and God save yourself, take upon my grandsire
